---
title: "Hough Transformation"
author: "Ryan E Lima"
email: Ryan.Lima@nau.edu
format: html
license: "CC BY 4.0"
---

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import sobel, gaussian_filter
import rasterio
import ipywidgets as widgets
from IPython.display import display
from matplotlib.colors import LightSource
import hyoga


AttributeError: module 'matplotlib.cm' has no attribute 'register_cmap'

In [ ]:
def plot_raster_histogram(array, title="Histogram", bins=100, log_scale=False, nodata=None):
    """
    Plot histogram of a raster array.

    Parameters:
        array (2D np.array): The raster data (e.g., DEM, hillshade).
        title (str): Title of the plot.
        bins (int): Number of bins in the histogram.
        log_scale (bool): If True, use log scale on y-axis.
        nodata (numeric or None): Value to ignore (e.g., -9999). If None, will ignore NaN.
    """
    # Mask NoData values
    if nodata is not None:
        array = np.ma.masked_equal(array, nodata)
    else:
        array = np.ma.masked_invalid(array)

    # Flatten and filter masked values
    data = array.compressed()

    plt.figure(figsize=(10, 5))
    plt.hist(data, bins=bins, edgecolor='black')
    plt.title(title)
    plt.xlabel("Pixel Value")
    plt.ylabel("Frequency")
    if log_scale:
        plt.yscale('log')
        plt.ylabel("Frequency (log scale)")
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_raster(array, title="Raster", cmap="terrain", vmin=None, vmax=None, colorbar=True):
    """
    Plots a DEM or hillshade raster array using matplotlib.

    Parameters:
        array (2D np.array): The raster to plot (DEM or hillshade).
        title (str): Plot title.
        cmap (str): Matplotlib colormap (e.g., 'terrain', 'gray').
        vmin (float): Minimum display value (optional).
        vmax (float): Maximum display value (optional).
        colorbar (bool): Whether to show colorbar.
    """
    plt.figure(figsize=(10, 6))
    img = plt.imshow(array, cmap=cmap, vmin=vmin, vmax=vmax)
    plt.title(title)
    plt.axis('off')
    if colorbar:
        plt.colorbar(img, shrink=0.7, label="Value")
    plt.tight_layout()
    plt.show()

In [ ]:

# Load and preprocess the DEM
raster_path = r"G:\Lineaments_python\LM1_10m_mshs.tiff"
with rasterio.open(raster_path) as src:
    mdhs = src.read(1).astype(float)
    nodata = src.nodata if src.nodata is not None else -9999

plot_raster_histogram(mdhs)



# Preview it
import matplotlib.pyplot as plt
plt.imshow(mdhs, cmap='viridis')
plt.title("MultiDirectional Hillshade")
plt.axis('off')
plt.show()




In [ ]:

# Optional smoothing and gradient
mdhs_smooth = gaussian_filter(mdhs, sigma=1)
gx = sobel(mdhs_smooth, axis=1)
gy = sobel(mdhs_smooth, axis=0)
grad_mag = np.hypot(gx, gy).astype(np.uint8)
mdhs_smooth = mdhs_smooth.astype(np.uint8)

plot_raster(mdhs_smooth)


In [ ]:
# Sliders for Canny thresholds
low_thresh = widgets.IntSlider(value=10, min=0, max=255, step=1, description='Low:')
high_thresh = widgets.IntSlider(value=50, min=1, max=255, step=1, description='High:')

# Ensure high threshold is always >= low threshold
def update_edges(low, high, array=mdhs_smooth):
    if low >= high:
        print("Low threshold must be less than high threshold.")
        return
    edge_output = cv2.Canny(array, low, high)

    plt.figure(figsize=(10, 6))
    plt.imshow(edge_output, cmap='gray')
    plt.title(f'Canny on Gradient Magnitude (Low={low}, High={high})')
    plt.axis('off')
    plt.show()

# Set up interactive display
ui = widgets.HBox([low_thresh, high_thresh])
out = widgets.interactive_output(update_edges, {'low': low_thresh, 'high': high_thresh})

display(ui, out)


In [ ]:
edges = cv2.Canny(grad_mag, 45, 60)

In [ ]:


# Plot the rasters in a 2x2 layout
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

# Plot dem_norm
axes[0, 0].imshow(mdhs, cmap='gray')
axes[0, 0].set_title("Multi-Directional Hillshade")
axes[0, 0].axis('off')

# Plot edges in red over dem_uint8
axes[0, 1].imshow(mdhs, cmap='gray')
axes[0, 1].imshow(edges, cmap='Reds', alpha=0.5)
axes[0, 1].set_title("MDHS with Edges")
axes[0, 1].axis('off')

# Plot dem_smooth
axes[1, 0].imshow(mdhs_smooth, cmap='viridis')
axes[1, 0].set_title("Smoothed mdhs")
axes[1, 0].axis('off')

# Plot grad_mag
axes[1, 1].imshow(grad_mag, cmap='viridis')
axes[1, 1].set_title("Gradient Magnitude")
axes[1, 1].axis('off')

# Add colorbars to all plots
for ax, data, cmap in zip(axes.flatten(), 
                          [dem_norm, dem_uint8, dem_smooth, grad_mag], 
                          ['grey', 'grey', 'viridis', 'viridis']):
    im = ax.images[0]  # Get the first image in the Axes
    fig.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()



In [ ]:


# Hough parameter sliders
threshold_slider = widgets.IntSlider(value=50, min=10, max=255, step=10, description='Threshold')
min_line_length_slider = widgets.IntSlider(value=30, min=5, max=200, step=5, description='Min Length')
max_line_gap_slider = widgets.IntSlider(value=5, min=0, max=50, step=1, description='Max Gap')

def hough_plot(threshold, min_line_length, max_line_gap):
    img_color = cv2.cvtColor(mdhs_smooth, cv2.COLOR_GRAY2BGR)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold,
                            minLineLength=min_line_length, maxLineGap=max_line_gap)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(img_color, (x1, y1), (x2, y2), (0, 255, 0), 2)

    plt.figure(figsize=(12, 10))
    plt.imshow(img_color)
    plt.title("Hough Lines Detected")
    plt.axis('off')
    plt.show()

# Build the interactive widget
ui = widgets.VBox([threshold_slider, min_line_length_slider, max_line_gap_slider])
out = widgets.interactive_output(hough_plot, {
    'threshold': threshold_slider,
    'min_line_length': min_line_length_slider,
    'max_line_gap': max_line_gap_slider
})

display(ui, out)


In [ ]:
import geopandas as gpd
from shapely.geometry import LineString
import rasterio

lines_list = []

# For geographic projection
with rasterio.open(raster_path) as src:
    transform = src.transform
    crs = src.crs




gdf = gpd.GeoDataFrame(geometry=lines_list, crs=crs)
# Extract contours from the edges
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Convert contours to LineString geometries
for contour in contours:
    if len(contour) > 1:  # Ensure the contour has at least two points
        line = LineString([tuple(pt[0]) for pt in contour])
        # Convert pixel coordinates to geographic coordinates
        line_geo = LineString([rasterio.transform.xy(transform, y, x, offset='center') for x, y in line.coords])
        lines_list.append(line_geo)

# Create GeoDataFrame and save to file
gdf = gpd.GeoDataFrame(geometry=lines_list, crs=crs)
gdf.to_file("detected_lineaments.shp")

In [ ]:
from shapely.geometry import LineString
import geopandas as gpd
import numpy as np

def explode_lines_to_segments(gdf):
    segments = []
    for line in gdf.geometry:
        coords = list(line.coords)
        for i in range(len(coords) - 1):
            segments.append(LineString([coords[i], coords[i+1]]))
    return gpd.GeoDataFrame(geometry=segments, crs=gdf.crs)


gdf_explode = explode_lines_to_segments(gdf)

In [ ]:
def calculate_orientations(gdf_segments):
    def azimuth(line):
        x1, y1, x2, y2 = *line.coords[0], *line.coords[1]
        angle = np.arctan2((x2 - x1), (y2 - y1)) * 180 / np.pi
        return angle % 180  # fold into 0–180°
    gdf_segments["azimuth"] = gdf_segments.geometry.apply(azimuth)
    return gdf_segments

In [ ]:
gdf_seg = calculate_orientations(gdf_explode)

In [ ]:
azimuths = gdf_seg['azimuth']

In [ ]:
from rasterio.plot import show
import geopandas as gpd

def plot_gdf_over_raster(raster_path, gdf_seg, ax=None, cmap="gray", gdf_kwargs={}):
    """
    Plot a GeoDataFrame on top of a georeferenced raster.

    Parameters:
        raster_path (str): Path to the GeoTIFF.
        gdf_seg (GeoDataFrame): Line segments to plot.
        ax (matplotlib axis): Optional existing axis.
        cmap (str): Colormap for the raster.
        gdf_kwargs (dict): Additional kwargs to pass to gdf_seg.plot().
    """
    with rasterio.open(raster_path) as src:
        fig, ax = plt.subplots(figsize=(10, 8)) if ax is None else (None, ax)
        show(src, ax=ax, cmap=cmap)
        
        # Reproject GeoDataFrame to match raster CRS if needed
        if gdf_seg.crs != src.crs:
            print("Reprojecting gdf_seg to match raster CRS...")
            gdf_seg = gdf_seg.to_crs(src.crs)
        
        # Plot segments
        gdf_seg.plot(ax=ax, **gdf_kwargs)

        ax.set_title("Segments over Raster")
        ax.axis('off')
        if fig is not None:
            plt.tight_layout()
            plt.show()

In [ ]:
# Make sure mdhs_smooth was saved to disk
raster_path = r"G:\Lineaments_python\LM1_10m_mshs.tiff"

plot_gdf_over_raster(
    raster_path=raster_path,
    gdf_seg=gdf_seg,
    gdf_kwargs={"color": "red", "linewidth": 1}
)

In [ ]:
from shapely.geometry import LineString
from shapely.ops import linemerge
from sklearn.neighbors import NearestNeighbors

def group_lines_by_azimuth_proximity(gdf, angle_thresh=10, dist_thresh=25):
    grouped = []
    used = set()

    coords = np.array([line.centroid.coords[0] for line in gdf.geometry])
    azimuths = gdf['azimuth'].values
    lines = gdf.geometry.values

    nbrs = NearestNeighbors(radius=dist_thresh).fit(coords)

    for i, (line_i, az_i) in enumerate(zip(lines, azimuths)):
        if i in used:
            continue

        group = [line_i]
        used.add(i)

        indices = nbrs.radius_neighbors([coords[i]], return_distance=False)[0]
        for j in indices:
            if j == i or j in used:
                continue
            az_j = azimuths[j]
            angle_diff = abs(az_i - az_j) % 180
            angle_diff = min(angle_diff, 180 - angle_diff)
            if angle_diff <= angle_thresh:
                group.append(lines[j])
                used.add(j)

        grouped.append(group)
    return grouped

In [ ]:
def bezier_curve_from_points(points, n_points=100):
    """
    Fit a Bézier curve to a list of (x, y) points using Bernstein polynomials.
    """
    points = np.array(points)
    n = len(points) - 1  # degree of Bézier

    def bernstein_poly(i, n, t):
        from scipy.special import comb
        return comb(n, i) * (t ** i) * ((1 - t) ** (n - i))

    t = np.linspace(0, 1, n_points)
    curve = np.zeros((n_points, 2))
    for i in range(n + 1):
        curve += np.outer(bernstein_poly(i, n, t), points[i])

    return LineString(curve)

In [ ]:
groups = group_lines_by_azimuth_proximity(gdf_seg, angle_thresh=10, dist_thresh=25)
smoothed_lines = []

for group in groups:
    # Gather all endpoints from the 2-point segments in the group
    pts = []
    for line in group:
        coords = list(line.coords)
        if not pts or pts[-1] != coords[0]:
            pts.extend(coords)
        else:
            pts.extend(coords[1:])

    # Remove duplicate points
    pts = [tuple(pt) for i, pt in enumerate(pts) if i == 0 or pt != pts[i-1]]
    
    if len(pts) < 3:
        smoothed_lines.append(LineString(pts))  # just connect
    else:
        bezier = bezier_curve_from_points(pts)
        smoothed_lines.append(bezier)

In [ ]:


gdf_bezier = gpd.GeoDataFrame(geometry=smoothed_lines, crs=gdf.crs)
gdf_bezier["length_m"] = gdf_bezier.geometry.length
min_length = 50  # meters
gdf_filtered = gdf_bezier[gdf_bezier["length_m"] >= min_length].copy()


In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))  # width, height in inches
gdf_filtered.plot(ax=ax, color='blue', linewidth=1)
plt.title("Smoothed Lineaments (Bézier)")
plt.axis('off')
plt.show()

In [ ]:
import warnings

def bspline_from_lines(line_group, smoothing=2, n_points=100):
    points = []
    for line in line_group:
        coords = list(line.coords)
        if not points or points[-1] != coords[0]:
            points.extend(coords)
        else:
            points.extend(coords[1:])

    if len(points) < 4:
        return LineString(points)

    x, y = zip(*points)

    with warnings.catch_warnings():
        warnings.simplefilter('ignore', category=RuntimeWarning)
        try:
            tck, u = splprep([x, y], s=smoothing)
            x_new, y_new = splev(np.linspace(0, 1, n_points), tck)
            return LineString(zip(x_new, y_new))
        except Exception:
            return LineString(points)  # fallback

In [ ]:
smoothed_lines = [bspline_from_lines(group) for group in groups]
gdf_smoothed = gpd.GeoDataFrame(geometry=smoothed_lines, crs=gdf.crs)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))  # width, height in inches
gdf_smoothed.plot(ax=ax, color='blue', linewidth=1)
plt.title("Smoothed Lineaments (Bézier)")
plt.axis('off')
plt.show()